In [1]:
import sys
sys.path.append("/Users/shikim/pynta_local/pynta/")

# Preprocessing notebook

## Preprocessing notebook will calculate: 
1. notes for choice of pseudopotentials
2. Lattice parameter optimization and analysis
3. Energy convergence test
4. k-points optimization

*Preprocessing can only run with surfaces that ASE can build*

In [ ]:
import os
import numpy as np
from ase.visualize import view
import matplotlib.pyplot as plt
from pynta.calculator import get_lattice_parameters
from pynta.preprocessing import *

### 1. Notes on Pseudopotentials:

We can choose from various pseudopotential libraries. Choice of pseudopotential depends on the problem we are investigating, *e.g.,* if there is a heavy element present in our system and we are interested in the spin-orbit coupling effects, we should choose a full relativistic pseudopotential. We need to be careful whether our chosen pseudopotential correctly reproduces physical properties. Various pseudopotential libraries:

- https://www.quantum-espresso.org/pseudopotentials
- https://www.materialscloud.org/discover/sssp/table/efficiency
- http://www.pseudo-dojo.org
- https://www.physics.rutgers.edu/gbrv/
- https://nninc.cnf.cornell.edu
- http://www.quantum-simulation.org/potentials/
- BLYP pseudopotentials (https://pseudopotentials.quantum-espresso.org/legacy_tables/hartwigesen-goedecker-hutter-pp) 
- SCAN pseudopotentials (https://yaoyi92.github.io/scan-tm-pseudopotentials.html)


### 2. Lattice parameter optimization:


If your slab is not optimized, it is NOT recommended to use for Pynta production run. Please optimize your slab before running Pynta

*Preprocessing can only run with surfaces that ASE can build*

In [3]:
#calculator
pseudo = {'Cu': 'Cu.pbe-spn-kjpaw_psl.1.0.0.UPF'}
command = '/Users/shikim/miniconda3/envs/pynta_env/bin/pw.x -in PREFIX.pwi > PREFIX.pwo'
software_kwargs={'command':command,'kpts': (10,10,10), 'ecutwfc': 70, 'degauss':0.02, 'mixing_mode': 'plain', 
                 'pseudopotentials':pseudo,'occupations':'smearing', 'smearing':'gauss',} 

In [ ]:
#calculate lattice constant using Pynta
get_lattice_parameters('Cu','fcc111','Espresso',software_kwargs)

In [ ]:
# test lattice constant
analyze_lattice_parameter('Cu','fcc111','Espresso',software_kwargs)

### 3. Cutoff Energy convergence test: 

We can calculate the total energy of the system for various values of energy cutoff values. 
Converged energy cutoff value with peudopotentials of choice can construct more accurate DFT input for Pynta. 

In [ ]:
prep = Prep(metal='Pt', surface_type='fcc111', a0=3.96, software='Espresso', software_kwargs={})
# Define your metal, software, software kwargs

In [ ]:
cutoff_energy = prep.opt_cutoff_energy(slab=None, ecut_range=(200, 501, 50),output_file='cutoff_energy.txt')

### 4. k-point convergence test:

With the optimized slab, appropriate k-points should be calculated to execute Pynta efficiently. 

Lowest total energy is estimated from lattice parameter optimization. We use the lowest potential energy to estimate k-points.

Tip: If you have already obtained satisfactory convergence with a (relatively) sparse k-point grid, there is no motivation to go for a denser grid. It makes calculations more expensive. 

In [ ]:
kpts_list, energies = prep.opt_kpoints(slab=None, kpts_range=[(1, 1, 1), (2, 2, 1)], output_file='kpts.txt')

In [ ]:
plot_results(cutoff_energy, kpts_list, energies)
# Plot energy vs energy cutoff 
# Plot energy vs k-points